In [ ]:
from PIL import Image
import cv2
import time
import copy
import numpy as np

input_path_2 = r"C:\Users\qianq\Videos\Captures\【楽譜あり】カナタハルカ_『すずめの戸締まり』より（ピアノソロ中級～上級）RADWIMPS【ピアノアレンジ楽譜】 - YouTube - Google Chrome 2023-03-10 17-24-00.mp4"

In [ ]:
#@梶尾さん
#input_path_2には、昨日slackに上げた一連の動画の一番上の動画をダウンロードして、そのパスを入れてください
#各鍵盤の真ん中のx座標は、上記の動画の場合の座標です。（なので、他の動画で試すと、座標の位置がその動画の鍵盤位置と合わなくなります）。本番では、鍵盤検出のコードの部分でこの座標をゲットします。
#出力は、最初の方は３桁の数字がいっぱい出ます。これは、毎回のバー認識の時に出したRGB値の差の合計です。
#出力の最後に、リストが出てきます。(inputした動画のスタートからの時間、音の高和の数字（midi形式にするのであれば、今回の場合は33足してください), on/off)の形式です。

reader = cv2.VideoCapture(input_path_2)
fps = reader.get(cv2.CAP_PROP_FPS)
#各鍵盤の真ん中のx座標
keyboards = [10, 34, 62, 91, 122, 151, 176, 207, 235, 262, 293, 318, 344, 372, 398, 427, 455, 486, 511, 538, 568, 594, 622, 654, 676, 706, 732, 760, 790, 820, 847, 874, 903, 932, 960, 986, 1012, 1040, 1067, 1097, 1123, 1156, 1181, 1209, 1237, 1267, 1293, 1321, 1349, 1375, 1400, 1429, 1459, 1489, 1515, 1544, 1572, 1601, 1629, 1655, 1681, 1708, 1734, 1764, 1795, 1824, 1851, 1879, 1909, 1932, 1963, 1988, 2019, 2044, 2069, 2098, 2132, 2156]
frame_midi = [(0, i, 0) for i in range(len(keyboards))] # 各フレームのmidi。
output_midi = [] #最終的に返すmidi
iteration = 0 #フレーム取得の繰り返し回数
frame_time = 0 #最初から数えた時の各フレームの経過時間
change_moment = [0 for i in range(len(keyboards))] #下と同じ
prev_change_moment = [0 for i in range(len(keyboards))] #とても短い間に２回急激な色変化が起きる場合に２回目を消す、という処理をするためのもの

while True:
    try:
        st = time.time()
        if iteration != 0:
            frame_time += 1/fps
        
        ret, frame = reader.read()
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame)
        
        current_pvs = []
        
        for i in range(len(keyboards)):
            pv = pil_image.getpixel((keyboards[i], 150)) #get pixel values
            current_pvs.append(pv)
            if iteration == 0:
                previous_pvs = current_pvs #previous_pvsの初期化
            
        for i in range(len(keyboards)):
            difference = abs(current_pvs[i][0] - previous_pvs[i][0]) + abs(current_pvs[i][1] - previous_pvs[i][1]) + abs(current_pvs[i][2] - previous_pvs[i][2])
            if difference >= 100:
                change_moment[i] = frame_time
                if change_moment[i] - prev_change_moment[i] > 0.1:
                    print(difference)
#                     plt.imshow(frame)
#                     plt.show()
                    if frame_midi[i][2] == 0:
                        frame_midi[i] = (frame_time, i, 1)
                    else:
                        frame_midi[i] = (frame_time, i, 0)
                    output_midi.append(frame_midi[i])
        
        prev_change_moment = copy.copy(change_moment)
        previous_pvs = current_pvs
        iteration += 1
        
        elapsed_time = time.time() - st
        if elapsed_time < 1/fps:
            time.sleep(1/fps - elapsed_time)
            
    except KeyboardInterrupt:
        break
        
reader.release()

print(output_midi)



#自分用のメモ
#はじめ、ピクセルRGB値の合計変化量の閾値を75にしたが、
#目視では色の変化が見当たらない時間帯にも、100~200位の色の変化が出る。（理由は不明）(憶測だが、バーの端っこの方だけ白→バーの色、みたいになってるのかも、細かすぎて人間の目では見えないが)
#よって、ピクセルRGB値の合計変化量の閾値を300に変更した

#上のやり方だと、動画によって都度閾値を変えないといけなく、非現実的なので、バーの端っこの色変化を避ける方法を変える
# とても短い時間の間で２回色の急激な変化があったら、２回目の方は消す。

# 残りの問題点
# timeが１秒ほどずれている
# 精度はそこそこあると思うが、正確に確認できていない